--- 
5.1 순서가 없는 범주형 특성 인코딩 

--- 

In [2]:
import numpy as np 
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer 

In [7]:
## 특성을 만들고, one-hot 형태로 인코딩 
features = np.array([['Texas'], 
                    ['California'], 
                    ['Texas'], 
                    ['Delaware'], 
                    ['Texas']])

one_hot = LabelBinarizer()
one_hot.fit_transform(features) 

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1]])

In [8]:
## Pandas를 사용한 one-hot 인코딩 
import pandas as pd

pd.get_dummies(features[:, 0]) 

,California,Delaware,Texas
0,0,0,1
1,1,0,0
2,0,0,1
3,0,1,0
4,0,0,1


In [11]:
## 다중 클래스 특성을 만들고, 0ne-hot 인코딩 하기 
multiclass_feature = [("Texas", "Florida"),
                      ("California", "Alabama"),
                      ("Texas", "Florida"),
                      ("Delware", "Florida"),
                      ("Texas", "Alabama")]

one_hot_multiclass = MultiLabelBinarizer()
one_hot_multiclass.fit_transform(multiclass_feature)

array([[0, 0, 0, 1, 1],
       [1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 1, 0],
       [1, 0, 0, 0, 1]])

In [13]:
## 사이킷런의 OneHotEncoder를 사용하면, 정수도 문자로 인식하여 처리한다 
from sklearn.preprocessing import OneHotEncoder 

feature_2 = np.array([['Texas', 1], 
                      ['California', 1], 
                      ['Texas', 3], 
                      ['Delaware', 1], 
                      ['Texas', 1]])

one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoder.fit_transform(feature_2)

array([[0., 0., 1., 1., 0.],
       [1., 0., 0., 1., 0.],
       [0., 0., 1., 0., 1.],
       [0., 1., 0., 1., 0.],
       [0., 0., 1., 1., 0.]])

--- 
5.2 순서가 있는 범주형 특서 인코딩 하기 

--- 

In [14]:
## 간단한 것은 pandas의 replace를 활용 
df = pd.DataFrame({"Score": ["Low", "Low", "Medium", "Medium", "High"]})
df

,Score
0,Low
1,Low
2,Medium
3,Medium
4,High


In [21]:
scale_mapper = {"Low" : 1,
                "Medium" : 2, 
                "High" : 3}

df['Score'].replace(scale_mapper)

0    1
1    1
2    2
3    2
4    3
Name: Score, dtype: int64

In [24]:
## 각 스케일 간격을 적절하게 조정할 수 있다. 
df_2 = pd.DataFrame({"Score": ["Low",
                                "Low",
                                "Medium",
                                "Medium",
                                "High",
                                "Barely More Than Medium"]})

scale_mapper = {"Low":1,
                "Medium":2,
                "Barely More Than Medium": 2.5,
                "High":3}

df_2["Score"].replace(scale_mapper)

0    1.0
1    1.0
2    2.0
3    2.0
4    3.0
5    2.5
Name: Score, dtype: float64

--- 
5.3 특성 딕셔너리를 인코딩 하기 

--- 

In [31]:
## 딕셔너리를 행렬로 변환할때 : 사이킷런의 DictVectorizer 사용 
from sklearn.feature_extraction import DictVectorizer 

data_dict = [{"Red": 2, "Blue": 4},
             {"Red": 4, "Blue": 3},
             {"Red": 1, "Yellow": 2},
             {"Red": 2, "Yellow": 2}]

dictvectorizer = DictVectorizer(sparse = False)
features = dictvectorizer.fit_transform(data_dict)
feature_names = dictvectorizer.get_feature_names() 

print(features) 
print(feature_names) 

[[4. 2. 0.]
 [3. 4. 0.]
 [0. 1. 2.]
 [0. 2. 2.]]
['Blue', 'Red', 'Yellow']


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [32]:
## 예시. 자연어 처리를 위해 4개의 문서에서 단어 - Red/Blue/Yellow -가 등장한 횟수를 행렬로 표현하기 

doc_1_word_count = {"Red": 2, "Blue": 4}
doc_2_word_count = {"Red": 4, "Blue": 3}
doc_3_word_count = {"Red": 1, "Yellow": 2}
doc_4_word_count = {"Red": 2, "Yellow": 2} 

doc_word_counts = [doc_1_word_count, doc_2_word_count, doc_3_word_count, doc_4_word_count]
doc_word_counts

[{'Blue': 4, 'Red': 2},
 {'Blue': 3, 'Red': 4},
 {'Red': 1, 'Yellow': 2},
 {'Red': 2, 'Yellow': 2}]

In [33]:
dictvectorizer = DictVectorizer(sparse = False)
dictvectorizer.fit_transform(doc_word_counts)

array([[4., 2., 0.],
       [3., 4., 0.],
       [0., 1., 2.],
       [0., 2., 2.]])

--- 
5.4 누락된 클래스 값 대체하기 

--- 

In [40]:
## 사이킷런을 이용, 결측치를 최빈값으로 대체하기 
from sklearn.impute import SimpleImputer 

X = np.array([[0, 2.10, 1.45],
              [1, 1.18, 1.33],
              [0, 1.22, 1.27],
              [1, -0.21, -1.19],
              [np.nan, 0.87, 1.31], 
              [np.nan, -0.67, -0.22]])
X

array([[ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19],
       [  nan,  0.87,  1.31],
       [  nan, -0.67, -0.22]])

In [42]:
imputer = SimpleImputer(strategy= 'most_frequent') ## strategy = 'most_frequent' , 'mean', 'median, ... 
imputer.fit_transform(X)

array([[ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19],
       [ 0.  ,  0.87,  1.31],
       [ 0.  , -0.67, -0.22]])

--- 
5.5 불균형한 클래스 다루기 (skipped) 

--- 